# Voronoi orbit binning
This testing notebook expects an existing all_models table and will act on its best-fit model.

In [ ]:
import dynamite as dyn

import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from vorbin.voronoi_2d_binning import voronoi_2d_binning

print('DYNAMITE version', dyn.__version__)
print('    installed at ', dyn.__path__)  # Uncomment to print the complete DYNAMITE installation path

fname = 'user_test_config_ml.yaml'
c = dyn.config_reader.Configuration(fname, reset_logging=True)
# c = dyn.config_reader.Configuration(fname, reset_logging=True, reset_existing_output=True)
# _ = dyn.model_iterator.ModelIterator(c)

In [ ]:
# instantiate a Coloring object
coloring = dyn.coloring.Coloring(c)

In [ ]:
# define the number of r and lambda_z bins
nr = 6
nl = 7

In [ ]:
plotter = dyn.plotter.Plotter(c)

In [ ]:
# get an overview of the orbit distribution
# NOTE: when using force_lambda_z=True, then the titles of the orbit-distribution plots are incorrect.
# All orbits are shown in this distribution - not only short-axis tubes!
fig1 = plotter.orbit_distribution(model=None, minr=None, maxr=None, r_scale='log', nr=nr, nl=nl,
                                  orientation='vertical', subset='short', force_lambda_z=True)
fig2 = plotter.orbit_distribution(model=None, minr=None, maxr=None, r_scale='linear', nr=nr, nl=nl,
                                  orientation='vertical', subset='short', force_lambda_z=True)

In [ ]:
# Perform Voronoi binning of orbits in the radius-circularity phase space. The goal is to group the "original"
# n_orbits orbit bundles into fewer n_bundle "Voronoi" bundles with each of these Voronoi bundles accounting for
# a weight of at least ``vor_weight``.
# The result is a tuple (vor_bundle_mapping, phase_space_binning):
#     vor_bundle_mapping : np.array of shape (n_bundle, n_orbits)
#         Mapping between the "original" orbit bundles and the Voronoi
#         orbit bundles: vor_bundle_mapping(i_bundle, i_orbit) is the
#         fraction of i_orbit assigned to i_bundle, multiplied by i_orbit's weight.
#     phase_space_binning : dict
#         'in': np.array of shape (3, nr*nl), the binning input:
#         bin r, bin lambda_z, bin total weight
#         'out': np.array of shape (3, n_bundle), the Voronoi binning output:
#         weighted Voronoi bin centroid coordinates r_bar, lambda_bar
#         and Voronoi bin total weights
#         'map': np.array of shape (nr*nl,) the phase space mapping:
#         Voronoi bin numbers for each input bin
vor_weight = 0.05  # define the desired (minimum) total orbital weight in each Voronoi bin
vor_bundle_mapping, phase_space_binning = coloring.bin_phase_space(model=None,
                                                                   minr='auto',
                                                                   maxr='auto',
                                                                   r_scale='log',
                                                                   nr=nr,
                                                                   nl=nl,
                                                                   vor_weight=vor_weight,
                                                                   vor_ignore_zeros=False,
                                                                   make_diagnostic_plots=True,
                                                                   extra_diagnostic_output=True,
                                                                   cvt=False,
                                                                   wvt=False)

In [ ]:
a = dyn.analysis.Analysis(c)

In [ ]:
bundle_maps = a.get_flux_for_selected_orbit_bundles(bundle_mapping=vor_bundle_mapping)

In [ ]:
# Plot maps for every Voronoi bundle
stars = c.system.get_unique_triaxial_visible_component()
map_plotter = stars.kinematic_data[0].get_map_plotter()
n_bundles = vor_bundle_mapping.shape[0]
fig = plt.figure(figsize=(30,30 / (n_bundles + 1) * 4))
fig.subplots_adjust(wspace=0.5)
for i, colname in enumerate(bundle_maps.colnames):
    ax = plt.subplot((n_bundles + 1) // 5 + (1 if n_bundles % 5 > 0 else 0), 5, i + 1)
    map_plotter(bundle_maps[colname], label='weight contr.', colorbar=True)
    ax.set_title(f'{colname}')
    ax.set_xlabel('x [arcsec]')
    ax.set_ylabel('y [arcsec]')
plt.show()

In [ ]:
# If dithering > 1, orbit bundles may contribute to multiple Voronoi orbit bundles
plt.figure(figsize=(24,4))
plt.gca().set_title('Weight that each orbit bundle contributes to Voronoi orbit bundles')
plt.pcolormesh(np.log10(vor_bundle_mapping), shading='nearest', cmap='Greys')
plt.xlabel('Orbit bundle id')
plt.ylabel('Voronoi orbit bundle id')
plt.colorbar(label='log Weight')

In [ ]:
# for each input bin: r coordinate, lambda_z coordinate, total weight
print(f'{phase_space_binning["in"].shape=}')
# phase_space_binning['in']

In [ ]:
# for each Voronoi bin: weighted centroid coordinates r_bar, lambda_bar and total weight
print(f'{phase_space_binning["out"].shape=}')
# phase_space_binning['out']

In [ ]:
# phase space mapping: Voronoi bin numbers for each input bin
print(f'{phase_space_binning["map"].shape=}')
phase_space_binning['map']

In [ ]:
# orbit bundle mapping: weighted contribution of each "original" orbit bundle to the Voronoi orbit bundles
print(f'{vor_bundle_mapping.shape=}')

## Testing the orbit binning
As dithering=1, each "original" orbit bundle is just one orbit. Hence, 100% of each such orbit bundle will lie in a unique $r, \lambda_z$ input bin and the `vor_bundle_mapping` will map these "original" orbit bundles to the Voronoi orbit bundles.

To test the orbit binning, we will - for each Voronoi bin (corresponding to one Voronoi orbit bundle each) - identify the connected "original" orbit bundles and add up their weighted fractions. These should be the same whether computed (a) from the binning input data, (b) from the total weights in the Voronoi bins, and (c) from adding all "original" orbit bundles' contributions in the `vor_bundle_mapping`.

In [ ]:
for vor_bin in range(phase_space_binning['out'].shape[-1]):  # do the following for each Voronoi bin

    input_bins = np.where(phase_space_binning['map']==vor_bin)[0]
    print(f'\ninput bins mapped to {vor_bin=}: {input_bins}')

    # orbit weights in input bins:
    weight_in = [phase_space_binning['in'][2][i] for i in input_bins]  # phase_space_binning['in'].shape=(3, nr*nl), holds r, l, w for each input bin
    weight_in_total = sum(weight_in)
    print(f'{weight_in=}, {weight_in_total=}')

    vorbin_weight = phase_space_binning['out'][2][vor_bin]  # phase_space_binning['out'].shape=(3, n_bundle), holds r, l, w for each Voronoi bin
    print(f'{vorbin_weight     = }')

    vor_bundle_weight = np.sum(vor_bundle_mapping[vor_bin])  # vor_bundle_mapping.shape=(n_bundle, n_orbits), contribution of orbit-weights to Voronoi bins
    print(f'{vor_bundle_weight = }')

    print('Test succeeded? ', np.isclose(weight_in_total, vorbin_weight) and np.isclose(weight_in_total, vor_bundle_weight))